In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Reshape, Attention, TimeDistributed, Embedding, LSTM, Bidirectional, Dropout, Dense, GRU, concatenate, Conv1D, Conv2D, Flatten,MaxPooling1D, MaxPooling2D, LocallyConnected1D, Activation, GaussianNoise, BatchNormalization, RepeatVector
from tensorflow.keras.models import Model, Sequential
import nltk
from nltk.corpus import stopwords
from AttentionWeightedAverage import AttentionWeightedAverage
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelBinarizer

- NLP on Title + Description to Tags, in order to build a tag recommendation algorithm
- NLP on Tags + Description + Category to Title, in order to provide title recommendation algorithm. These videos are popular videos, meaning they have appealing titles. Let's build a system, that judging by content, can provide the most appropriate title.

In [12]:
df = pd.read_csv("./data/USvideos.csv")
print('Number of youtube videos in dataframe: ', len(df.index))
df.head()

# count category occurences
occurences = df['category_id'].value_counts()
# category names
map_categories = {}
with open('data/US_category_id.json') as json_file:
    data_category = json.load(json_file)
    for category in data_category['items']:
        int_id = int(category['id'])
        occ = 0
        if int_id in occurences:
            occ = occurences[int_id]
        map_categories[int_id] = [category['snippet']['title'],occ]
# create new dataframe with categories and appearances
df_categories = pd.DataFrame.from_dict(map_categories, orient='index', columns=['category_name', 'count'])
# add category name on the main dataframe
for i in map_categories.keys():
    df.loc[df['category_id'] == i,'category_name'] = map_categories[i][0]

Number of youtube videos in dataframe:  40949


### Tags, Description and Category -> Auto generate Title
We will see a word level seq2seq model

In [13]:
def discretize(tags):
    tags = tags.replace('"','')
    discrete_tags = tags.split('|')
    return discrete_tags

In [14]:
df_text = df[['title', 'description', 'tags', 'category_name']]
df_text = df_text.dropna(axis=0, how='any')

titles = df_text['title'].tolist()
descriptions = df_text['description'].tolist()
tags = [discretize(row['tags']) for index, row in df_text.iterrows()]
categories = df_text['category_name'].tolist()

- Titles need to be tokenized, lowercased, categorical.
- Descriptions need to be tokenized, lowercased, categorical.
- Tags need to be lowercased, categorical.

In [15]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [16]:
stops = set(stopwords.words("english"))

In [28]:
clean_desc = []
clean_title = []
for desc in descriptions:
    desc = desc.replace('\\n', ' ')
    desc = desc.lower()
    desc = desc.replace('www.facebook.com', 'FacebookLink')
    desc = desc.replace('http.facebook.com', 'FacebookLink')
    desc = desc.replace('www.twitter.com', 'TwitterLink')
    desc = desc.replace('www.amazon.com', 'AmazonLink')
    desc = desc.replace('http.amzn.com', 'AmazonLink')
    desc = desc.replace('www.instagram.com', 'InstagramLink')
    desc = desc.replace('www.snapchat.com', 'SnapchatLink')
    desc = desc.replace('https://bit.ly', 'PortalLink')
    temp = text_to_word_sequence(desc, filters='!"#%&()*-+,.\\/:;<=>?@[\\]^_`{|}~\t\n♡▶➜', lower=True, split=' ')
    for w in temp:
        if w in stops:
            temp.remove(w)
    clean_desc.append(temp)
    
for title in titles:
    title = title.replace('\\n', ' ')
    title = title.replace('\xa0', ' ')
    seq = text_to_word_sequence(title, filters='"#%&()*-+,.\\/:;<=>?@[\\]^_`{|}~\t\n♡▶➜', lower=True, split=' ')
    clean_title.append(['START_ '] + seq + [' _END'])

Training an unsupervised method for word2vec with gensim, in order to provide me with very good embeddings

In [21]:
from gensim.models import Word2Vec
input_features = 100

gensim_model = Word2Vec(size=input_features,
                       window=5,
                       min_count=1,
                       sg=1,
                       hs=0,
                       negative=10,
                       workers=4)
gensim_model.build_vocab(clean_desc+clean_title+tags)
gensim_model.train(clean_desc+clean_title+tags, total_examples=len(clean_desc+clean_title+tags), epochs=10)

(57580053, 64013890)

In [29]:
all_title_words=set()
for title in clean_title:
    for word in title:
        if word not in all_title_words:
            all_title_words.add(word)

In [30]:
num_encoder_tokens = len(gensim_model.wv.vocab)

target_words = sorted(list(all_title_words))
num_decoder_tokens = len(all_title_words)

Creating my new sequences with vector representations of words

In [31]:
X_train_description = []
X_train_category = []
X_train_tags = []

lb_cat = LabelBinarizer()
lb_cat.fit(list(set(categories)))

maxLenDesc = 0
for desc in clean_desc:
    X_train_description.append(np.asarray([gensim_model.wv[word] for word in desc]))
    if len(desc) > maxLenDesc:
        maxLenDesc = len(desc)
        
maxLenTags = 0
for taglist in tags:
    X_train_tags.append(np.asarray([gensim_model.wv[tag] for tag in taglist]))
    if len(taglist) > maxLenTags:
        maxLenTags = len(taglist)

for cat in categories:
    X_train_category.append(lb_cat.transform([cat]))

maxLenTitle = 0
for title in clean_title:
    if len(title) > maxLenTitle:
        maxLenTitle = len(title)

Create the decoder inputs and outputs. Decoder target data will be ahead by one timestep, and will not include the start character.

In [32]:
X_train_title = np.zeros(
    (len(clean_title), maxLenTitle, input_features),
    dtype='float32')
Y_train_title = np.zeros(
    (len(clean_title), maxLenTitle, num_decoder_tokens),
    dtype='float32')

# generate data
for i, target_text in enumerate(clean_title):
    for t, word in enumerate(target_text):
        # decoder input data, will have the embeddings of the words
        X_train_title[i, t] = gensim_model.wv[word]
        if t > 0:
            # decoder target data, will be ahead by one timestep, and will contain
            # one hot representations of what word is selected
            Y_train_title[i, t - 1, target_words.index(word)] = 1.

MemoryError: Unable to allocate 37.9 GiB for an array with shape (40379, 22, 11444) and data type float32

In [ ]:
# padding
X_train_description = pad_sequences(X_train_description, maxlen=maxLenDesc, padding='post')
X_train_tags = pad_sequences(X_train_tags, maxlen=maxLenTags, padding='post')

# convert to numpy arrays
X_train_description = np.stack(X_train_description, axis=0)
X_train_category = np.stack(X_train_category, axis=0)
X_train_tags = np.stack(X_train_tags, axis=0)

### Sequence to sequence model

In [ ]:
def Seq2SeqModel(vocab_size,input_max_seq,output_target_seq,input_features):
    #encoder
    encoder_input = Input(shape=(input_max_seq,input_features), name='main_input')
    encoder_lstm = LSTM(100, return_state=True, name='encoder_lstm')
    encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_input)
    encoder_states = [encoder_state_h, encoder_state_c]
    
    #decoder
    decoder_input = Input(shape=(None,input_features))
    decoder_lstm = LSTM(100, return_sequences=True, return_state=True, name='decoder_lstm')
    decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_input, initial_state=encoder_states)
    
    decoder_dense = Dense(vocab_size, name='decoder_dense', activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    
    model = Model(inputs=[encoder_input, decoder_input], outputs=decoder_outputs)
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=4e-3), metrics=['accuracy'])
    # encoder model
    encoder_model = Model(encoder_input, encoder_states)

    decoder_state_inputs = [Input(shape=(100,)), Input(shape=(100,))]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_input, initial_state=decoder_state_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_input] + decoder_state_inputs, [decoder_outputs] + decoder_states)
    
    return model,encoder_model,decoder_model

In [ ]:
epochs=1
batch_size=64
model,encoder_model,decoder_model = Seq2SeqModel(num_decoder_tokens,maxLenDesc,maxLenTitle,100)
model.summary()

In [ ]:
model.fit([X_train_description, X_train_title], Y_train_title, epochs=epochs, batch_size=batch_size,verbose=1)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or len(decoded_sentence) > maxLenTitle):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
print (decode_sequence(X_train_desc[22]))